In [248]:
from urllib import request
import pandas as pd
import datetime as dt


In [308]:
URLS = {"Confirmed":
        "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv",
        "Recovered":
        'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv',
        "Deaths":
        'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv', 
        }

for url_key in URLS:
    print ("- Downloading", url_key, "...")
    request.urlretrieve(URLS[url_key], './input/{}_cases.csv'.format(url_key))
print ("- Done!")


confirmed = pd.read_csv('./input/Confirmed_cases.csv')
recovered  =recv_df = pd.read_csv('./input/Recovered_cases.csv')
deaths = pd.read_csv('./input/Deaths_cases.csv')

confirmed.head()

- Downloading Confirmed ...
- Downloading Recovered ...
- Downloading Deaths ...
- Done!


,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,3/23/20,3/24/20,3/25/20,3/26/20,3/27/20,3/28/20,3/29/20,3/30/20,3/31/20,4/1/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,40,74,84,94,110,110,120,170,174,237
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,...,104,123,146,174,186,197,212,223,243,259
2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,0,...,230,264,302,367,409,454,511,584,716,847
3,NaN,Andorra,42.5063,1.5218,0,0,0,0,0,0,...,133,164,188,224,267,308,334,370,376,390
4,NaN,Angola,-11.2027,17.8739,0,0,0,0,0,0,...,3,3,3,4,4,5,7,7,7,8


In [311]:
# Getting all dates
all_dates = confirmed.columns[4:]

new_confirmed = confirmed.melt(id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], 
                            value_vars=all_dates, var_name='Date', value_name='Confirmed')

new_recovered = recovered.melt(id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], 
                            value_vars=all_dates, var_name='Date', value_name='Recovered')

new_deaths = deaths.melt(id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], 
                            value_vars=all_dates, var_name='Date', value_name='Deaths')

clean_data = pd.concat([new_confirmed, new_recovered['Recovered'], new_deaths['Deaths']], axis=1)


# removing county wise data to avoid double counting
# clean_data = clean_data[clean_data['Province/State'].str.contains(',')!=True]

clean_data['Date'] = pd.to_datetime(clean_data['Date'])

clean_data.head()

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Recovered,Deaths
0,NaN,Afghanistan,33.0000,65.0000,2020-01-22,0,0.0,0
1,NaN,Albania,41.1533,20.1683,2020-01-22,0,0.0,0
2,NaN,Algeria,28.0339,1.6596,2020-01-22,0,0.0,0
3,NaN,Andorra,42.5063,1.5218,2020-01-22,0,0.0,0
4,NaN,Angola,-11.2027,17.8739,2020-01-22,0,0.0,0


In [312]:
clean_data["Country/Region"].replace({'US': 'United States'}, inplace=True)
clean_data["Country/Region"].replace({'Czechia': 'Czech Republic'}, inplace=True)
clean_data["Country/Region"].replace({'Korea, South': 'South Korea'}, inplace=True)

In [313]:
clean_data["Date"] = pd.to_datetime(clean_data["Date"])

In [314]:
clean_data

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Recovered,Deaths
0,NaN,Afghanistan,33.000000,65.000000,2020-01-22,0,0.0,0
1,NaN,Albania,41.153300,20.168300,2020-01-22,0,0.0,0
2,NaN,Algeria,28.033900,1.659600,2020-01-22,0,0.0,0
3,NaN,Andorra,42.506300,1.521800,2020-01-22,0,0.0,0
4,NaN,Angola,-11.202700,17.873900,2020-01-22,0,0.0,0
...,...,...,...,...,...,...,...,...
18171,Turks and Caicos Islands,United Kingdom,21.694000,-71.797900,2020-04-01,6,NaN,0
18172,NaN,MS Zaandam,0.000000,0.000000,2020-04-01,9,NaN,2
18173,NaN,Botswana,-22.328500,24.684900,2020-04-01,4,NaN,1
18174,NaN,Burundi,-3.373100,29.918900,2020-04-01,2,NaN,0


# Fatality Rate por dia

In [315]:
df_fatality_rate = pd.read_csv("./input/coronavirus-cfr.csv", parse_dates=['Date'], infer_datetime_format=True)

In [316]:
df_fatality_rate.rename(columns={'Entity': 'Country', 'Case fatality rate of COVID-19 (%) (Only observations with ≥100 cases) (%)': 'Case fatality rate of COVID-19 (%)'}, inplace=True)

In [317]:
df_fatality_rate = df_fatality_rate[['Country', 'Date', 'Case fatality rate of COVID-19 (%)']]

In [318]:
df_fatality_rate.head()

,Country,Date,Case fatality rate of COVID-19 (%)
0,Afghanistan,2020-03-29,2.830189
1,Afghanistan,2020-03-30,3.508772
2,Afghanistan,2020-03-31,2.836879
3,Afghanistan,2020-04-01,2.409639
4,Albania,2020-03-24,4.000000


# Daily Confirmed 

In [319]:
df_daily_confirmed = pd.read_csv("./input/daily-covid-cases-3-day-average.csv", parse_dates=['Date'], infer_datetime_format=True)

In [320]:
df_daily_confirmed.rename(columns={'Entity': 'Country'}, inplace=True)

In [321]:
df_daily_confirmed = df_daily_confirmed[['Country', 'Date', 'Daily new confirmed cases of COVID-19 (rolling 3-day average)']]

In [322]:
df_daily_confirmed.head()

,Country,Date,Daily new confirmed cases of COVID-19 (rolling 3-day average)
0,Afghanistan,2019-12-31,0.0
1,Afghanistan,2020-01-01,0.0
2,Afghanistan,2020-01-02,0.0
3,Afghanistan,2020-01-03,0.0
4,Afghanistan,2020-01-04,0.0


# Daily Deaths 

In [323]:
df_daily_death = pd.read_csv("./input/daily-covid-deaths-3-day-average.csv", parse_dates=['Date'], infer_datetime_format=True)

In [324]:
df_daily_death.rename(columns={'Entity': 'Country'}, inplace=True)

In [325]:
df_daily_death = df_daily_death[['Country', 'Date', 'Daily new confirmed deaths due to COVID-19 (rolling 3-day average)']]

In [326]:
df_daily_death.head()

,Country,Date,Daily new confirmed deaths due to COVID-19 (rolling 3-day average)
0,Afghanistan,2019-12-31,0.0
1,Afghanistan,2020-01-01,0.0
2,Afghanistan,2020-01-02,0.0
3,Afghanistan,2020-01-03,0.0
4,Afghanistan,2020-01-04,0.0


# Adicionando informações das cidades (NY, São Paulo, Lombardia e Wuhan)

## Hubei - Wuhan

In [327]:
df_hubei = clean_data[(clean_data['Province/State'] == 'Hubei')].reset_index()

In [328]:
df_hubei.rename(columns={'Province/State': 'Country'}, inplace=True)

In [329]:
df_hubei = df_hubei[['Country', 'Date', 'Confirmed', 'Recovered', 'Deaths']]

In [330]:
df_hubei['Date'] = pd.to_datetime(df_hubei['Date'])

In [331]:
df_hubei.replace('Hubei','Wuhan', inplace=True)

In [332]:
df_hubei.sort_values('Date', inplace=True)

daily_cases = df_hubei.Confirmed.diff().fillna(0)

df_hubei["Daily new confirmed deaths due to COVID-19 (rolling 3-day average)"] = daily_cases.rolling(3).mean().fillna(0)

daily_cases = df_hubei.Deaths.diff().fillna(0)
df_hubei["Daily new confirmed cases of COVID-19 (rolling 3-day average)"] = daily_cases.rolling(3).mean().fillna(0)

In [333]:
df_hubei.head()

,Country,Date,Confirmed,Recovered,Deaths,Daily new confirmed deaths due to COVID-19 (rolling 3-day average),Daily new confirmed cases of COVID-19 (rolling 3-day average)
0,Wuhan,2020-01-22,444,0.0,17,0.000000,0.000000
1,Wuhan,2020-01-23,444,0.0,17,0.000000,0.000000
2,Wuhan,2020-01-24,549,0.0,24,35.000000,2.333333
3,Wuhan,2020-01-25,761,0.0,40,105.666667,7.666667
4,Wuhan,2020-01-26,1058,0.0,52,204.666667,11.666667


## Lombardia

In [334]:
df_lombardia = pd.read_csv('./input/covid19_italy_region.csv', infer_datetime_format=True, parse_dates=['Date'])

In [335]:
df_lombardia['Date'] = pd.to_datetime(df_lombardia['Date']).dt.date

In [336]:
df_lombardia = df_lombardia[df_lombardia['RegionName'] == 'Lombardia']

In [337]:
df_lombardia = df_lombardia[['RegionName', 'Date', 'TotalPositiveCases', 'Recovered', 'Deaths']]

In [338]:
df_lombardia.rename(columns={'RegionName': 'Country', 'TotalPositiveCases':'Confirmed'}, inplace=True)

In [339]:
df_lombardia.reset_index(inplace=True)

In [340]:
df_lombardia.sort_values('Date', inplace=True)
daily_cases = df_lombardia.Confirmed.diff().fillna(0)

df_lombardia["Daily new confirmed deaths due to COVID-19 (rolling 3-day average)"] = daily_cases.rolling(3).mean().fillna(0)

daily_cases = df_lombardia.Deaths.diff().fillna(0)
df_lombardia["Daily new confirmed cases of COVID-19 (rolling 3-day average)"] = daily_cases.rolling(3).mean().fillna(0)

## NYC e SP

In [341]:
df_nyc_sp = pd.read_csv('./input/nysp_new.csv', infer_datetime_format=True, parse_dates=['Date'])

In [342]:
df_nyc_sp = df_nyc_sp[['Country', 'Date', 'Confirmed', 'Deaths']]

In [343]:
df_nyc = df_nyc_sp[df_nyc_sp['Country'] == 'New York'].reset_index()
df_sp = df_nyc_sp[df_nyc_sp['Country'] == 'São Paulo'].reset_index()

In [344]:
df_nyc.sort_values('Date', inplace=True)

daily_cases = df_nyc.Confirmed.diff().fillna(0)

df_nyc["Daily new confirmed deaths due to COVID-19 (rolling 3-day average)"] = daily_cases.rolling(3).mean().fillna(0)

daily_cases = df_nyc.Deaths.diff().fillna(0)
df_nyc["Daily new confirmed cases of COVID-19 (rolling 3-day average)"] = daily_cases.rolling(3).mean().fillna(0)


df_sp.sort_values('Date', inplace=True)

daily_cases = df_sp.Confirmed.diff().fillna(0)

df_sp["Daily new confirmed deaths due to COVID-19 (rolling 3-day average)"] = daily_cases.rolling(3).mean().fillna(0)

daily_cases = df_sp.Deaths.diff().fillna(0)
df_sp["Daily new confirmed cases of COVID-19 (rolling 3-day average)"] = daily_cases.rolling(3).mean().fillna(0)

## Juntando todas as cidades

In [345]:
df_hubei.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 71 entries, 0 to 70
Data columns (total 7 columns):
 #   Column                                                              Non-Null Count  Dtype         
---  ------                                                              --------------  -----         
 0   Country                                                             71 non-null     object        
 1   Date                                                                71 non-null     datetime64[ns]
 2   Confirmed                                                           71 non-null     int64         
 3   Recovered                                                           67 non-null     float64       
 4   Deaths                                                              71 non-null     int64         
 5   Daily new confirmed deaths due to COVID-19 (rolling 3-day average)  71 non-null     float64       
 6   Daily new confirmed cases of COVID-19 (rolling 3-day average)

In [346]:
df_cities = pd.concat([df_nyc, df_sp, df_lombardia, df_hubei])

In [347]:
df_cities.reset_index(inplace=True)

In [348]:
df_cities['Case fatality rate of COVID-19 (%)'] = df_cities['Deaths'] * 100 / df_cities['Confirmed'] 

In [349]:
df_cities = df_cities [['Country', 'Date', 'Confirmed', 'Deaths', "Daily new confirmed deaths due to COVID-19 (rolling 3-day average)", "Daily new confirmed cases of COVID-19 (rolling 3-day average)", 'Case fatality rate of COVID-19 (%)']]

In [350]:
df_cities

,Country,Date,Confirmed,Deaths,Daily new confirmed deaths due to COVID-19 (rolling 3-day average),Daily new confirmed cases of COVID-19 (rolling 3-day average),Case fatality rate of COVID-19 (%)
0,New York,2020-03-10 00:00:00,25,0,0.000000,0.000000,0.000000
1,New York,2020-03-11 00:00:00,55,0,0.000000,0.000000,0.000000
2,New York,2020-03-12 00:00:00,95,0,23.333333,0.000000,0.000000
3,New York,2020-03-13 00:00:00,154,0,43.000000,0.000000,0.000000
4,New York,2020-03-14 00:00:00,269,1,71.333333,0.333333,0.371747
...,...,...,...,...,...,...,...
146,Wuhan,2020-03-28 00:00:00,67801,3177,0.000000,4.666667,4.685772
147,Wuhan,2020-03-29 00:00:00,67801,3182,0.000000,4.333333,4.693146
148,Wuhan,2020-03-30 00:00:00,67801,3186,0.000000,4.000000,4.699046
149,Wuhan,2020-03-31 00:00:00,67801,3187,0.000000,3.333333,4.700521


# Juntando todos os dados

In [351]:
clean_data.rename(columns={'Country/Region': 'Country'}, inplace=True)

In [352]:
clean_data = clean_data.groupby(["Country", "Date"])[['Confirmed', 'Deaths', 'Recovered']].sum().reset_index()

In [354]:
clean_data = pd.merge(clean_data, df_fatality_rate, left_on=['Country', 'Date'], right_on=['Country', 'Date'], how='inner', suffixes=('', ''))
clean_data = pd.merge(clean_data, df_daily_confirmed, left_on=['Country', 'Date'], right_on=['Country', 'Date'], how='inner', suffixes=('', ''))
clean_data = pd.merge(clean_data, df_daily_death, left_on=['Country', 'Date'], right_on=['Country', 'Date'], how='inner', suffixes=('', ''))

clean_data.head()

,Country,Date,Confirmed,Deaths,Recovered,Case fatality rate of COVID-19 (%),Daily new confirmed cases of COVID-19 (rolling 3-day average),Daily new confirmed deaths due to COVID-19 (rolling 3-day average)
0,Afghanistan,2020-03-29,120,4,5.0,2.830189,13.000000,1.000000
1,Afghanistan,2020-03-30,170,4,0.0,3.508772,16.666667,0.666667
2,Afghanistan,2020-03-31,174,4,0.0,2.836879,20.000000,0.333333
3,Afghanistan,2020-04-01,237,4,0.0,2.409639,26.000000,0.000000
4,Albania,2020-03-24,123,5,0.0,4.000000,15.666667,1.000000


In [355]:
df_final = pd.concat([clean_data, df_cities])

In [356]:
df_final.head()

,Country,Date,Confirmed,Deaths,Recovered,Case fatality rate of COVID-19 (%),Daily new confirmed cases of COVID-19 (rolling 3-day average),Daily new confirmed deaths due to COVID-19 (rolling 3-day average)
0,Afghanistan,2020-03-29 00:00:00,120,4,5.0,2.830189,13.000000,1.000000
1,Afghanistan,2020-03-30 00:00:00,170,4,0.0,3.508772,16.666667,0.666667
2,Afghanistan,2020-03-31 00:00:00,174,4,0.0,2.836879,20.000000,0.333333
3,Afghanistan,2020-04-01 00:00:00,237,4,0.0,2.409639,26.000000,0.000000
4,Albania,2020-03-24 00:00:00,123,5,0.0,4.000000,15.666667,1.000000


In [357]:
df_final.to_csv("./input/covid_19_clear.csv", index=False)